<p style="text-align: center; font-size: 200%; font-weight: bold; margin-top: 10px;">CVI5 Python - Auditorias = Resumo</p>

## 🚧 Ajuste inicial de sys.path para a base de cvi5py

In [1]:
import sys
import os
msg = "Corrigido o primeiro item de sys.path de " + sys.path[0]
cvi5path = os.getcwd()
path_parts = cvi5path.split(os.sep)
# por padrão, srcP\cvi5 está no nível três
cvi5path = os.sep.join(path_parts[:3])
cvi5path = os.path.join(cvi5path, 'py')
sys.path.remove(sys.path[0])
sys.path.insert(0, cvi5path)
print(msg, " para ", sys.path[0])

Corrigido o primeiro item de sys.path de J:\srcP\cvi5\py\notebooks  para  J:\srcP\cvi5\py


## ✅ Imports

In [2]:
import sqlite3
import pandas as pd
import numpy as np
from pycvi.Config import Config
from pycvi.CviDb import CviDb
from pycvi.audit.AuditHelpers import AuditHelpers
from IPython.display import display, HTML

## ⤴️ OSF selecionada em cvi5.py

In [3]:
config = Config()
ah = AuditHelpers(config)
cvi_sys_data = config.load_cvi_sys_data()
cv_db = ah.audits_action(cvi_sys_data['osf_atual'], cvi_sys_data['cnpj_atual'])

sql = """
SELECT max(nome) AS Empresa, max(cnpj) AS CNPJ, max(ie) AS IE FROM
    (SELECT nome, cnpj, ie FROM _fiscal_participantedeclarado WHERE idParticipanteDeclarado = 1)
"""
df = pd.read_sql_query(sql, cv_db.conn)
display(df.head())

,Empresa,CNPJ,IE
0,CREATIVE DISPLAY INDUSTRIA E COMERCIO LTDA,02175187000190,379152700113


### Bloco GIA/EFD

In [4]:
sql = """
 SELECT 
  B.cnpj, B.ie,  substr(B.dataDeReferencia, 1, 4) || substr(B.dataDeReferencia, 6, 2) AS aaaamm, B.dataDeEntrega, B.idTipoDeOperacaoNaGia AS tpGia, 
  B.campo51SaidasComDebito AS c51, B.campo56EntradasComCredito AS c56, round(B.campo51SaidasComDebito - B.campo56EntradasComCredito, 2) AS sdoOper,
  B.campo52OutrosDebitos AS c52, B.campo53EstornoDeCreditos AS c53, B.campo57OutrosCreditos AS c57, B.campo58EstornoDeDebitos AS c58, 
  round(B.campo52OutrosDebitos + B.campo53EstornoDeCreditos - B.campo57OutrosCreditos - B.campo58EstornoDeDebitos, 2) AS sdoNOper,
  B.campo55TotalDeDebitos AS c55, B.campo60SubtotalDeCreditos AS c60,
  B.campo61SaldoCredorDoPeriodoAnterior AS c61, B.campo62TotalDeCreditos AS c62, B.campo63SaldoDevedor AS c63, B.campo64Deducoes AS c64, B.campo65ImpostoARecolher AS c65,
  B.campo66SaldoCredorATransportar AS c66 
 FROM
(SELECT  cnpj, ie, idTipoDeOperacaoNaGia, dataDeReferencia, max(dataDeEntrega) AS dataDeEntrega
   FROM _fiscal_ApuracaoDeIcmsPelaGia AS A
   GROUP BY cnpj, ie, idTipoDeOperacaoNaGia, dataDeReferencia) AS sqA
LEFT OUTER JOIN _fiscal_ApuracaoDeIcmsPelaGia AS B ON B.cnpj = sqA.cnpj AND B.ie = sqA.ie AND B.idTipoDeOperacaoNaGia = sqA.idTipoDeOperacaoNaGia AND B.dataDeReferencia =  sqA.dataDeReferencia  AND  B.dataDeEntrega = sqA.dataDeEntrega;
"""
df_gia = pd.read_sql_query(sql, cv_db.conn)
df_gia.head().style.pipe(ah.df_style)

,cnpj,ie,aaaamm,dataDeEntrega,tpGia,c51,c56,sdoOper,c52,c53,c57,c58,sdoNOper,c55,c60,c61,c62,c63,c64,c65,c66
0,02175187000190,379152700113,201901,2019-04-10,"0,00","294.616,55","156.889,18","137.727,37","1.299,20","0,00","865,06","0,00","434,14","295.915,75","157.754,24","0,00","157.754,24","138.161,51","0,00","138.161,51","0,00"
1,02175187000190,379152700113,201902,2019-04-10,"0,00","486.599,64","188.539,69","298.059,95","974,85","0,00","1.688,72","0,00","-713,87","487.574,49","190.228,41","0,00","190.228,41","297.346,08","0,00","297.346,08","0,00"
2,02175187000190,379152700113,201903,2019-04-22,"0,00","581.501,28","77.629,40","503.871,88","1.664,39","0,00","1.503,68","0,00","160,71","583.165,67","79.133,08","0,00","79.133,08","504.032,59","0,00","504.032,59","0,00"
3,02175187000190,379152700113,201904,2019-05-17,"0,00","411.494,10","173.464,86","238.029,24","356,15","0,00","1.192,27","7.517,60","-8.353,72","411.850,25","182.174,73","0,00","182.174,73","229.675,52","0,00","229.675,52","0,00"
4,02175187000190,379152700113,201905,2019-06-24,"0,00","497.416,83","242.928,97","254.487,86","0,00","0,00","0,00","0,00","0,00","497.416,83","242.928,97","0,00","242.928,97","254.487,86","0,00","254.487,86","0,00"


In [5]:
# SELECIONADA SOMENTE A MAIS RECENTE DATA DE GIA ENTREGUE
# Se houver mais de uma GIA entregue na mesma data mais recente, as duas serão listadas
# Concentre a análise nos campos sdoOper e sdoNOper, especialmente quando negativo. Analise também c65 e c66
df_gia[['sdoOper','sdoNOper','c65']].sum()

sdoOper     6080080.30
sdoNOper    -193612.29
c65         5886468.01
dtype: float64

### Bloco Madf

In [6]:
sql = """
SELECT
  g1, 
  sum(valconGia) AS valconGia, sum(valconEfd) AS valconEfd, sum(valconDif) AS valconDif,
  sum(icmsGia) AS icmsGia, sum(icmsEfd) AS icmsEfd, sum(icmsDif) AS icmsDif,
  sum(icmsstGia) AS icmsstGia, sum(icmsstEfd) AS icmsstEfd, sum(icmsstDif) AS icmsstDif
  FROM madf
  GROUP BY g1
UNION ALL
SELECT '#Período de ' || min(aaaamm) ||  ' a ' || max(aaaamm) ||  '#', Null, Null, Null, Null, Null, Null, Null, Null, Null
  FROM madf
UNION ALL
SELECT 'Repeticao de Ref e CFOP: ' || 
  CASE WHEN max(qtd) > 0 THEN '#' || max(qtd) || '#' ELSE max(qtd) END
  AS maxqtd, Null, Null, Null, Null, Null, Null, Null, Null, Null FROM
(SELECT CFOP, referencia, count(referencia) AS qtd  FROM
_fiscal_ComparacaoGiaEfdPorCfop
GROUP BY CFOP, referencia) AS sqA
ORDER BY g1;
"""
query = cv_db.conn.execute(sql)
cols = [column[0] for column in query.description]

df_madfgroup = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
df_madfgroup.style.pipe(ah.df_style)

,g1,valconGia,valconEfd,valconDif,icmsGia,icmsEfd,icmsDif,icmsstGia,icmsstEfd,icmsstDif
0,#Período de 201901 a 202012#,nan,nan,nan,nan,nan,nan,nan,nan,nan
1,1-Receitas,"78.346.664,47","78.344.773,22","1.891,25","8.835.396,55","8.835.396,55","0,00","0,00","0,00","0,00"
2,2-Compras Insumos,"-25.155.982,31","-25.195.787,49","39.805,18","-2.838.693,70","-2.846.062,69","7.368,99","0,00","0,00","0,00"
3,3-Compras Consumo,"-3.499.563,05","-3.448.587,34","-50.975,71","1.866,68","1.866,68","0,00","0,00","0,00","0,00"
4,4-Ativo,"-3.259.061,82","-3.302.296,82","43.235,00","0,00","0,00","0,00","0,00","0,00","0,00"
5,5-Entradas/Saídas,"40.975.101,27","40.971.559,02","3.542,25","136.861,98","136.861,98","0,00","0,00","0,00","0,00"
6,6-Outros,"1.366.455,50","1.341.958,28","24.497,22","-55.351,21","-55.333,21","-18,00","0,00","0,00","0,00"
7,Repeticao de Ref e CFOP: #1#,nan,nan,nan,nan,nan,nan,nan,nan,nan


In [7]:
sql = """
SELECT * FROM madf;
"""
query = cv_db.conn.execute(sql)
cols = [column[0] for column in query.description]
df_madf = pd.DataFrame.from_records(data = query.fetchall(), columns = cols)
df_madf_g = df_madf.groupby(["g1"])
df_madf_g.aggregate({'valconGia': 'sum', 'valconEfd': 'sum', 'valconDif': 'sum',
               'icmsGia': 'sum', 'icmsEfd': 'sum', 'icmsDif': 'sum',
               'icmsstGia': 'sum', 'icmsstEfd': 'sum', 'icmsstDif': 'sum'}).style.pipe(ah.df_style)

,valconGia,valconEfd,valconDif,icmsGia,icmsEfd,icmsDif,icmsstGia,icmsstEfd,icmsstDif
g1,,,,,,,,,
1-Receitas,"78.346.664,47","78.344.773,22","1.891,25","8.835.396,55","8.835.396,55","0,00","0,00","0,00","0,00"
2-Compras Insumos,"-25.155.982,31","-25.195.787,49","39.805,18","-2.838.693,70","-2.846.062,69","7.368,99","0,00","0,00","0,00"
3-Compras Consumo,"-3.499.563,05","-3.448.587,34","-50.975,71","1.866,68","1.866,68","0,00","0,00","0,00","0,00"
4-Ativo,"-3.259.061,82","-3.302.296,82","43.235,00","0,00","0,00","0,00","0,00","0,00","0,00"
5-Entradas/Saídas,"40.975.101,27","40.971.559,02","3.542,25","136.861,98","136.861,98","0,00","0,00","0,00","0,00"
6-Outros,"1.366.455,50","1.341.958,28","24.497,22","-55.351,21","-55.333,21","-18,00","0,00","0,00","0,00"


### Bloco ChaveNroTudao

In [8]:
df_chaveNroTudao = pd.read_sql_query("SELECT * FROM chaveNroTudao;", cv_db.conn)
df_chaveNroTudao.head()

,codSit,indEmit,indOper,ref,chave,numero,uf,dif_vl_doc,dif_icms,dif_icmsstSP,...,EfdPartValsCons,EfdPartValCon,EfdPartIcmss,EfdPartIcms,DFeNatOp,DFeCodncms,EfdCodncms,DFeDescris,EfdDescris,obs
0,2,0,1,2019-09,35190902175187000190550010000313221357458874,31322,MS,448262.83,31378.42,0.0,...,None,NaN,None,NaN,VENDA DE PROD. ESTABELECIMENTO,39205900|94032000|94036000,39205900|94032000|94036000,106670 - ROLLER 450019132|110475 - PLATAFORMA ...,106670 - ROLLER 450019132|110475 - PLATAFORMA ...,- \n
1,2,0,1,2020-03,35200302175187000190550010000326271449532326,32627,SP,362405.29,43488.64,0.0,...,None,NaN,None,NaN,VENDA DE PROD. DO ESTABELECIMENTO *,94032000,94032000,DISPLAY DE CHAO TRADICION|DISPLAY DE CHAO TRAD...,DISPLAY DE CHAO TRADICION|DISPLAY DE CHAO TRAD...,"- Base Legal: Artigo 54, inciso XIII, alinea ..."
2,2,0,1,2019-10,35191002175187000190550010000315471343510852,31547,SP,315999.97,37920.00,0.0,...,None,NaN,None,NaN,VENDA A ORDEM*,94037000,94037000,KIT BERGERIE MOVEL CONCEITO 1200 MM - REMERCH ...,KIT BERGERIE MOVEL CONCEITO 1200 MM - REMERCH ...,- NOTA FISCAL EMITIDA CONFORME PARAGRAFO 2 IT...
3,2,0,1,2019-09,35190902175187000190550010000313211225168529,31321,PA,304072.91,21285.11,0.0,...,None,NaN,None,NaN,VENDA DE PROD. ESTABELECIMENTO,94032000,94032000,106670 - ROLLER 450019132|110476 - BANCO CALCA...,106670 - ROLLER 450019132|110476 - BANCO CALCA...,- \n
4,2,0,1,2019-11,35191102175187000190550010000318021634377502,31802,SP,222750.00,26730.00,0.0,...,None,NaN,None,NaN,VENDA DE PROD. DO ESTABELECIMENTO *,94037000,94037000,KIT BERGERIERS MOVEL PANVEL 600 MM,KIT BERGERIERS MOVEL PANVEL 600 MM,"- Base Legal: Artigo 54, inciso XIII, alinea ..."


In [9]:
df_chaveNroTudao2 = df_chaveNroTudao[['codSit','vl_docDFe', 'vl_docEFD', 'vl_icmsDFe', 'vl_icmsEFD']]
df_chaveNroTudao2_g = df_chaveNroTudao2.groupby(["codSit"])
df_chaveNroTudao2_g.describe().style.pipe(ah.df_style)

In [10]:
# Creating new columns with formulas
df_chaveNroTudao['aliqCalcDFe'] = df_chaveNroTudao['vl_icmsDFe'] / df_chaveNroTudao['vl_docDFe']
# Creating new columns with if/then logic
df_chaveNroTudao['tpValorDFe'] = np.where(df_chaveNroTudao['vl_docDFe'] > 200000, "Alto", "Baixo")
df_chaveNroTudao2 = df_chaveNroTudao[['codSit','vl_docDFe', 'vl_docEFD', 'vl_icmsDFe', 'vl_icmsEFD', 'aliqCalcDFe', 'tpValorDFe']]
# Sorting by values
df_chaveNroTudao2 = df_chaveNroTudao2.sort_values(['vl_docDFe'], ascending=False)
# filtering ( [df_chaveNroTudao2['vl_docDFe'] > 100000] )
df_chaveNroTudao2[df_chaveNroTudao2['vl_docDFe'] > 100000].head().style.pipe(ah.df_style)
## TODO
##    PROCV (merge)
##    PIVOT TABLE (pivot_table)

,codSit,vl_docDFe,vl_docEFD,vl_icmsDFe,vl_icmsEFD,aliqCalcDFe,tpValorDFe
4175,<>2-3-4-5,"2.060.286,54","2.060.286,54","0,00","0,00","0,00",Alto
1006,<>2-3-4-5,"1.599.999,98","1.599.999,98","192.000,00","192.000,00","0,12",Alto
6076,<>2-3-4-5,"768.973,69","768.973,69","0,00","0,00","0,00",Alto
4297,<>2-3-4-5,"660.133,24","660.133,24","79.215,99","79.215,99","0,12",Alto
4571,<>2-3-4-5,"617.524,06","617.524,06","74.102,89","74.102,89","0,12",Alto


  - Abaixo, demonstração de um styler cheio de coisas ( https://pandas.pydata.org/docs/user_guide/style.html )

In [11]:
weather_df = pd.DataFrame(np.random.rand(10,2)*5,
                          index=pd.date_range(start="2021-01-01", periods=10),
                          columns=["Tokyo", "Beijing"])

def rain_condition(v):
    if v < 1.75:
        return "Dry"
    elif v < 2.75:
        return "Rain"
    return "Heavy Rain"

def make_pretty(styler):
    styler.set_caption("Weather Conditions")
    styler.format(rain_condition)
    styler.applymap(lambda v: 'color:red;' if v < 2 else None)
    styler.format_index(lambda v: v.strftime("%A"))
    styler.background_gradient(axis=None, vmin=1, vmax=5, cmap="YlGnBu")
    return styler

# weather_df
weather_df.loc["2021-01-04":"2021-01-08"].style.pipe(make_pretty)

,Tokyo,Beijing
Monday,Heavy Rain,Heavy Rain
Tuesday,Rain,Heavy Rain
Wednesday,Dry,Heavy Rain
Thursday,Heavy Rain,Rain
Friday,Rain,Heavy Rain
